In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torchvision

from tqdm import tqdm


In [4]:
datatrain = r'D:\Data\train\train'
datatest = r'D:\Data\test'
masktrain = r'D:\Data\train_gt\train_gt'

In [ ]:
# #load data by cv2

# images_train=[]
# images_test=[]
# masks_train=[]

# for filename in os.listdir(datatrain):
#     img = cv2.imread(os.path.join(datatrain,filename))
#     if img is not None:
#         images_train.append(img)

# for filename in os.listdir(datatest):
#     img = cv2.imread(os.path.join(datatest,filename))
#     if img is not None:
#         images_test.append(img)

# for filename in os.listdir(masktrain):
#     img = cv2.imread(os.path.join(masktrain,filename))
#     if img is not None:
#         masks_train.append(img)

In [ ]:
#load data by torchvision with OOP
class PolypDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.img_names = os.listdir(img_dir)
    
    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        mask_path = os.path.join(self.mask_dir, self.img_names[idx])

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)

        if self.transform:
            augment = self.transform(image=image, mask=mask)
            image = augment["image"]
            mask = augment["mask"]

        return image, mask


In [1]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
    A.Resize(1024, 1024),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.05,
        rotate_limit=20,
        p=0.5
    ),
    A.GaussianBlur(p=0.2),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(1024, 1024),
    A.Normalize(),
    ToTensorV2(),
])


ModuleNotFoundError: No module named 'albumentations'

# Model Architecture

In [ ]:
class U_net(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super.__init__()